In [1]:
#-- The current version is V1.2
#-- This is updated from v1.1 to V1.2
#-- The updated version changes the nonth calculation to the scientific average for month. i.e. 30.4 days is equals 1 month.
#-- Updated by Yusupha Ceesay on 29/07/2024
#-- Implemented on 01/08/2024
#-------------------------------------------------------------------------#

In [2]:
#-- The current version is V1.1
#-- This is updated from v1.0 to V1.1
#-- The updated version is a refactored code of the previous version, and including the biographic data of the participants.
#-- Updated by Yusupha Ceesay on 29/07/2024
#-- Implemented on 01/08/2024
#-------------------------------------------------------------------------#

In [3]:
from redcap import Project
import pandas as pd
import datetime
today=pd.Timestamp.today()

#REDCap API configuration
##project configuration function
def project_configuration(api_url,api_key):
    return Project(api_url,api_key)

###indigo project configuration
project = project_configuration('https://ruff.mrc.gm:8443/redcap/api/', 'D6807C091B1FC14F57CC9B2972D0B398')

###Sensitization project configuration
sen_project = project_configuration('https://redcap.mrc.gm:8443/redcap/api/','B5C8DD01C47A66E81E154A8AC735D281')


In [4]:
##Village mapping
village_mapping = {
'001':'Dumbuto',
'002':'Sankandi',
'003':'Nioro Jattaba',
'004':'Jattaba',
'005':'Jiffarong',
'006':'Bajana',
'007':'Kuli Kunda',
'008':'Jamaru',
'009':'Brikamanding',
'010':'Kantong Kunda',
'011':'Jali',
'013':'Manduar',
'014':'Bang Kuling',
'015':'Gissay',
'016':'Tankular',
'017':'Joli',
'018':'Kuyang',
'019':'Bantasu',
'020':'Santamba',
'021':'Missira',
'022':'Taborangkoto',
'023':'Burong',
'024':'Jula Kunda',
'025':'Karantaba',
'026':'Mandina',
'027':'Janneh Kunda',
'028':'Kemoto',
'029':'Keneba',
'030':'Batelling',
'031':'Sandeng',
'032':'Wudeba',
'034':'Kenokoto',
'035':'Manari',
'036':'Nineteen',
'040':'WUROKANG',
'041':'KWINELLA SANSANKONO',
'042':'KWINELLA NIA KUNDA',
'043':'TENDABA',
'044':'BUMARR',
'045':'BAMBAKO',
'046':'KUNDONG MARIAYA',
'047':'NEMA',
'048':'KUNDANG NUMU KUNDA',
'049':'KUNDANG FULA KUNDA',
'050':'NEMA KUTA',
'051':'JIRROFF',
'052':'MADINA ANGALLEH',
'053':'JATTA KUNDA',
'054':'MANDINA CENTRAL',
'055':'SARE SARJO',
'056':'SIBETO',
'057':'SARE NDALLA',
'058':'TABANANI',
'060':'WILLINGARA',
'061':'SARE MAMUDU'
}


In [5]:
# #Function to process sensitization data
# def process_sensitization_data(project,village_mapping):
#     fields = ['in_kiang_number', 'in_name', 'in_age', 'in_village', 'in_compound', 
#         'in_eden_number', 'in_contact1', 'in_contact2', 'in_contact3', 'in_contact4', 
#         'in_outcome', 'in_study_number'
#         ]
#     #sen_log_data= export_records_to_df(project,['indigo_sensitisation'],fields)
#     sen_log_data=project.export_records(forms=['indigo_sensitisation'])
#     sen_log_data = sen_log_data[(sen_log_data['in_outcome']=='1') &
#                                 (sen_log_data['in_kiang_number'].notnull()) &
#                                 (sen_log_data['in_study_number'].notnull())
#                                 ]
#     sen_log_data=sen_log_data.rename(columns={'in_study_number': 'con_participantid_q1'})
#     sen_log_data['in_village'] = sen_log_data['in_village'].map(village_mapping)

#     return sen_log_data

In [6]:
#Function to process sensitization data

def process_sensitization_data(project, village_mapping):
    fields = ['in_kiang_number', 'in_name', 'in_age', 'in_village', 'in_compound', 
              'in_eden_number', 'in_contact1', 'in_contact2', 'in_contact3', 'in_contact4', 
              'in_outcome', 'in_study_number']
    
    # Export records from the project
    sen_log_data = project.export_records(forms=['indigo_sensitisation'])
    
    # Convert to DataFrame if necessary
    if isinstance(sen_log_data, list):
        sen_log_data = pd.DataFrame(sen_log_data)

    # Filter the data
    sen_log_data = sen_log_data[(sen_log_data['in_outcome'] == '1') &
                                (sen_log_data['in_kiang_number'].notnull()) &
                                (sen_log_data['in_study_number'].notnull())]
    
    # Rename columns
    sen_log_data = sen_log_data.rename(columns={'in_study_number': 'con_participantid_q1'})
    
    # Map village names
    sen_log_data['in_village'] = sen_log_data['in_village'].map(village_mapping)

    return sen_log_data


In [7]:
#SENSITIZATION DATA
sen_log_data=process_sensitization_data(sen_project,village_mapping)

In [8]:

# Function to load and convert data from REDCap forms
def load_redcap_data(form_name):
    data = project.export_records(forms=[form_name])
    return pd.DataFrame(data)


In [9]:
# Load data from the relevant forms
MatSup_data = load_redcap_data('maternal_supplementation')
Rand_data = load_redcap_data('randomization')
inf_dob = load_redcap_data('delivery')
inf_dt = load_redcap_data('infantid')
term_data = load_redcap_data('study_termination')

In [10]:

# Process Randomization data
Rand_data = Rand_data[['con_participantid_q1', 'redcap_event_name', 'ran_date', 'rnd_arm']]
Rand_data['ran_date'] = pd.to_datetime(Rand_data['ran_date'], errors='coerce')

# Mapping randomization groups
rnd_group = {
    '1': 'B', '2': 'N', '3': 'P', '4': 'A', '5': 'X', '6': 'Y',
    '7': 'C', '8': 'U', '9': 'E', '10': 'K', '11': 'J', '12': 'H',
    '13': 'W', '14': 'T', '15': 'L'
}
rnd_data = Rand_data[(Rand_data['redcap_event_name'] == 'week_20_arm_1') & (Rand_data['rnd_arm'] != "")]
rnd_data['rnd_arm'] = rnd_data['rnd_arm'].map(rnd_group)

# Process Delivery data
inf_del = inf_dob[['con_participantid_q1', 'del_ddate_q7', 'redcap_event_name']]
inf_del['del_ddate_q7'] = pd.to_datetime(inf_del['del_ddate_q7'], errors='coerce')
inf_del = inf_del[(inf_del['redcap_event_name'] == 'delivery__postpart_arm_1') & inf_del['del_ddate_q7'].notnull()]

# Process Infant ID data
inf_dt = inf_dt[['con_participantid_q1', 'infantid', 'redcap_event_name']]
inf_dt = inf_dt[inf_dt['redcap_event_name'] == 'baby_check_arm_2']

# Merge infant and mother delivery data
inf_mother_dob = pd.merge(inf_dt, inf_del, on='con_participantid_q1', how='inner')
inf_data = inf_mother_dob[['con_participantid_q1', 'infantid', 'del_ddate_q7']]
df_infant = pd.DataFrame(inf_data)
df_infant['del_ddate_q7'] = pd.to_datetime(df_infant['del_ddate_q7'], errors='coerce')

# Calculate the end date based on 6 months (considering 1 month = 30.4 days)
df_infant['end_date'] = df_infant['del_ddate_q7'] + pd.to_timedelta(6 * 30.4, unit='D')

# Merge infant data with randomization data
Rand1 = rnd_data[['con_participantid_q1', 'ran_date', 'rnd_arm']]
randomized_part = pd.merge(df_infant, Rand1, on='con_participantid_q1', how='right')
randomised = randomized_part[['con_participantid_q1', 'infantid', 'ran_date', 'rnd_arm', 'del_ddate_q7', 'end_date']]

# Process Withdrawals data
df_withdrw = term_data[['con_participantid_q1', 'st_date', 'st_reason', 'redcap_event_name']]
df_withdrw['st_date'] = pd.to_datetime(df_withdrw['st_date'], errors='coerce')
df_withdrawals = df_withdrw[(df_withdrw['redcap_event_name'] == 'end_of_study_arm_1') & df_withdrw['st_date'].notnull()]

# Exclude withdrawals from randomized data
ran_con_merged_df = pd.merge(randomised, df_withdrawals, on='con_participantid_q1', how='inner')

# Filter out participants who withdrew
randomised = randomised[~randomised['con_participantid_q1'].isin(ran_con_merged_df['con_participantid_q1'])]

# Format date to have ony date and no time.
randomised['ran_date'] = randomised['ran_date'].dt.date
randomised['del_ddate_q7'] = randomised['del_ddate_q7'].dt.date
randomised['end_date'] = randomised['end_date'].dt.date

C:\Users\abliebah\AppData\Local\Temp\ipykernel_5420\1810256095.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rnd_data['rnd_arm'] = rnd_data['rnd_arm'].map(rnd_group)
C:\Users\abliebah\AppData\Local\Temp\ipykernel_5420\1810256095.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inf_del['del_ddate_q7'] = pd.to_datetime(inf_del['del_ddate_q7'], errors='coerce')
C:\Users\abliebah\AppData\Local\Temp\ipykernel_5420\1810256095.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [11]:
# Function to merge with sensitization data
def merge_with_sensitization(sup_df, sensitization_df):
    return pd.merge(sup_df, sensitization_df, on='con_participantid_q1', how='left')


In [12]:
# Merge with sensitization data
df_randomised = merge_with_sensitization(randomised, sen_log_data)

In [13]:

randomised_part= pd.DataFrame({
        'PARTICIPANT ID': df_randomised['con_participantid_q1'],
        'INFANT ID': df_randomised['infantid'],
        'KIANG NUMBER': df_randomised['in_kiang_number'],
        'NAME': df_randomised['in_name'],
        'VILLAGE': df_randomised['in_village'],
        'COMPOUND': df_randomised['in_compound'],
        'CONTACT1': df_randomised['in_contact1'],
        'CONTACT2': df_randomised['in_contact2'],
        'CONTACT3': df_randomised['in_contact3'],
        'CONTACT4': df_randomised['in_contact4'],
        'Randomization Date': df_randomised['ran_date'],
        'Randomization Arm': df_randomised['rnd_arm'],
       'Infant DoB': df_randomised['del_ddate_q7'],
    'Supplementation end date': df_randomised['end_date'],
})



In [14]:
# Function to save DataFrame to CSV with a timestamp
def save_to_csv(df, base_filename):
    now = datetime.datetime.now()
    timestamp = now.strftime('%Y%m%d_%H%M%S')
    filename = f'{base_filename}_{timestamp}.csv'
    df.to_csv(filename, index=False)


In [15]:
# Save the to a CSV file
save_to_csv(randomised_part, 'SupplementatonReport_v1.2')